## US CS Faculty Dataset
- [CS Faculty Composition and Hiring Trends (Blog)](https://jeffhuang.com/computer-science-open-data/#cs-faculty-composition-and-hiring-trends)
- [2200 Computer Science Professors in 50 top US Graduate Programs](https://cs.brown.edu/people/apapouts/faculty_dataset.html)
- [CS Professors (Data Explorer)](https://drafty.cs.brown.edu/csprofessors?src=csopendata)
- [Drafty Project](https://drafty.cs.brown.edu/)


Use beautiful_soup to scrap CS Faculty info

In [1]:
from scrap_cs_faculty import *

In [2]:
SCHOOL = "MIT-AID"
URL = SCHOOL_DICT[SCHOOL]["url"]  #  "https://www.eecs.mit.edu/role/faculty-aid/"
print(URL)

https://www.eecs.mit.edu/role/faculty-aid/


In [3]:
page = requests.get(URL, headers=BROWSER_HEADERS)

In [4]:
soup = BeautifulSoup(page.content, "html.parser")

## Find Elements by HTML Class Name

In [5]:
cs_persons = soup.find_all("div", class_="people-entry small-12 medium-6 large-4 larger-3 cell")

In [6]:
len(cs_persons) , cs_persons[0]

(92,
 <div class="people-entry small-12 medium-6 large-4 larger-3 cell">
 <a aria-hidden="true" class="people-index-image" href="https://www.eecs.mit.edu/people/hal-abelson/" rel="bookmark" tabindex="-1" title="Hal Abelson">
 <img alt="Hal Abelson" class="attachment-people-thumb size-people-thumb wp-post-image" decoding="async" height="228" loading="lazy" sizes="(max-width: 228px) 100vw, 228px" src="https://www.eecs.mit.edu/wp-content/uploads/2021/06/halab.png" srcset="https://www.eecs.mit.edu/wp-content/uploads/2021/06/halab.png 228w, https://www.eecs.mit.edu/wp-content/uploads/2021/06/halab-125x125.png 125w" width="228"/> </a>
 <h5><a href="https://www.eecs.mit.edu/people/hal-abelson/" rel="bookmark">Hal Abelson</a></h5>
 <p>Class of 1992 Professor , [CS and AI+D]</p>
 <ul>
 <li><a href="mailto:hal@mit.edu">hal@mit.edu</a></li>
 <li>(617) 253-5856</li>
 <li>Office: 32-G516</li>
 </ul>
 <div class="people-research">
 <p><a href="?fwp_research=ai-and-society">AI and Society</a></p><p><

### Extract Text From HTML Elements

You can add .text to a Beautiful Soup object to return only the text content of the HTML elements that the object contains:

In [7]:
DEBUG =  False # True # 

school, dept = map_school_dept(SCHOOL)
data = []
all_research_dict = {}
for n, person in enumerate(cs_persons):
    try:
        data_dict = {"school": school, "department": dept} # default
        if DEBUG and n > 0: break  # debug
        
        # get name/url
        x = person.find("h5")
        name_x = x.find("a")
        data_dict['name'] = name_x.text.strip()
        data_dict['url'] = name_x["href"]
        
        # get img
        x = person.find("img")
        if x:
            data_dict['img_url'] = x["src"]
            if "http" not in data_dict['img_url']:
                y = person.find("a")
                data_dict['img_url'] = y["href"]
                if not data_dict['img_url']:
                    data_dict['img_url'] = data_dict['url']

        research_area_dict = {}
        for x in person.find_all("p"):
            if (is_job_title(x.text.lower())) and "[" in x.text:
                # get title and dept
                x_tmp = x.text.split("[")
                if len(x_tmp) > 0:
                    title = x_tmp[0].strip()
                    data_dict['job_title'] = title[:-1] if title.endswith(",") else title
                if len(x_tmp) > 1:
                    depts = [i.strip() for i in x_tmp[1].replace("]", "").split("and") if i.strip()]
                    if DEBUG: print(f"depts = {depts}")
                    depts = [DEPT_MAP.get(d, "") for d in depts]
                    data_dict['department'] = "; ".join(depts)
            else:
                # get research
                y = x.find("a")
                if y:
                    research_name = y.text.strip()
                    if research_name:
                        research_area_dict[research_name] = f"{URL}{y['href']}"
                        if research_name not in all_research_dict:
                            all_research_dict[research_name] = research_area_dict[research_name]
        researchs = research_area_dict.keys()
        if researchs:
            data_dict['research_area'] = ", ".join(researchs)
                
        # get email, phone, office
        for x in person.find_all("li"):
            if "Office" in x.text.strip():
                data_dict['office_address'] = x.text.strip()
            else:
                y = x.find("a")
                if y and "@" in y.text:
                    data_dict['email'] = y.text.strip()
                else:
                    data_dict['phone'] = x.text.strip()
                
        if DEBUG:
            print(f"n={n}\t=============")
            print(f"name= {data_dict.get('name','')}")
            print(f"job_title= {data_dict.get('job_title','')}")
            print(f"phone= {data_dict.get('phone','')}")
            print(f"office= {data_dict.get('office_address','')}")
            print(f"email= {data_dict.get('email','')}")
            print(f"url= {data_dict.get('url','')}")
            print(f"img_url= {data_dict.get('img_url','')}")
            print(f"research_area= {data_dict.get('research_area','')}")
            print(f"department= {data_dict.get('department','')}")
        
        if data_dict:
            row_data = []
            for c in COLUMNS:
                cell = data_dict.get(c,"")
                row_data.append(cell)
            data.append(row_data)
    except Exception as e:
        print(f"[Error] {str(e)}\n{person.prettify()}")

In [8]:
len(data), data[-1]

(92,
 ['Victor Zue',
  'Delta Electronics Professor of EECS (Post-Tenure)',
  '',
  '',
  'Educational Technology, Human-Computer Interaction, Natural Language and Speech Processing',
  '',
  '',
  'https://www.eecs.mit.edu/people/victor-zue/',
  'https://www.eecs.mit.edu/people/victor-zue/',
  '(617) 253-8513',
  'zue@csail.mit.edu',
  '',
  'Office: 32-G470',
  'AI & Decision-making; Computer Science',
  'Massachusetts Institute Technology'])

In [9]:
df = pd.DataFrame(data, columns=COLUMNS)

In [10]:
print(f"Number of faculties at {SCHOOL}: {df.shape[0]}")

Number of faculties at MIT-AID: 92


In [11]:
df

,name,job_title,phd_univ,phd_year,research_area,research_concentration,research_focus,url,img_url,phone,email,cell_phone,office_address,department,school
0,Hal Abelson,Class of 1992 Professor,,,"AI and Society, Artificial Intelligence + Deci...",,,https://www.eecs.mit.edu/people/hal-abelson/,https://www.eecs.mit.edu/wp-content/uploads/20...,(617) 253-5856,hal@mit.edu,,Office: 32-G516,Computer Science; AI & Decision-making,Massachusetts Institute Technology
1,Elfar Adalsteinsson,"Education Officer for Electrical Engineering, ...",,,"AI for Healthcare and Life Sciences, Biologica...",,,https://www.eecs.mit.edu/people/elfar-adalstei...,https://www.eecs.mit.edu/people/elfar-adalstei...,(617) 324-3597,elfar@mit.edu,,Office: 36-766,Electrical Engineering; AI & Decision-making,Massachusetts Institute Technology
2,Pulkit Agrawal,Steven and Renee Finn Career Development Profe...,,,"Artificial Intelligence + Machine Learning, Gr...",,,https://www.eecs.mit.edu/people/pulkit-agrawal/,https://www.eecs.mit.edu/people/pulkit-agrawal/,(617) 253-5851,pulkitag@mit.edu,,Office: 32-342,AI & Decision-making; Computer Science,Massachusetts Institute Technology
3,Mohammad Alizadeh,Associate Professor,,,"Multicore Processors & Cloud Computing, Securi...",,,https://www.eecs.mit.edu/people/mohammad-aliza...,https://www.eecs.mit.edu/people/mohammad-aliza...,(617) 253-6042,alizadeh@mit.edu,,Office: 32-G920,Computer Science;,Massachusetts Institute Technology
4,Jacob Andreas,"X-Window Consortium Professor, Assistant Profe...",,,"Artificial Intelligence + Machine Learning, Na...",,,https://www.eecs.mit.edu/people/jacob-andreas/,https://www.eecs.mit.edu/people/jacob-andreas/,617-253-0352,jda@mit.edu,,Office: 32-386H,AI & Decision-making,Massachusetts Institute Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Ashia Wilson,Lister Brothers (Gordon K. ’30 and Donald K. ’...,,,"AI and Society, Artificial Intelligence + Mach...",,,https://www.eecs.mit.edu/people/ashia-wilson/,https://www.eecs.mit.edu/wp-content/uploads/20...,,ashia07@mit.edu,,Office: 32-D660,AI & Decision-making,Massachusetts Institute Technology
88,Gregory Wornell,Sumitomo Electric Industries Professor in Engi...,,,"Artificial Intelligence + Machine Learning, Co...",,,https://www.eecs.mit.edu/people/gregory-wornell/,https://www.eecs.mit.edu/people/gregory-wornell/,(617) 253-3513,gww@mit.edu,,Office: 36-677,AI & Decision-making,Massachusetts Institute Technology
89,Guangyu Robert Yang,Assistant Professor / Shared appointment in BCS,,,Artificial Intelligence + Decision making,,,https://www.eecs.mit.edu/people/guangyu-robert...,https://www.eecs.mit.edu/wp-content/uploads/20...,(475) 201-7714,yanggr@mit.edu,,Office: 46,AI & Decision-making,Massachusetts Institute Technology
90,Lizhong Zheng,Professor of EE,,,"Communications Systems, Information Science an...",,,https://www.eecs.mit.edu/people/lizhong-zheng/,https://www.eecs.mit.edu/people/lizhong-zheng/,(617) 452-2941,lizhong@mit.edu,,Office: 36-660,AI & Decision-making; Electrical Engineering,Massachusetts Institute Technology


In [12]:
# prepare research group dataframe
cols = ["research_group", "url"]
data = []
for i in all_research_dict.keys():
    data.append([i, all_research_dict.get(i,'')])
    print(f"{i}:\t {all_research_dict.get(i,'')}")

AI and Society:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=ai-and-society
Artificial Intelligence + Decision making:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=artificial-intelligence-decision-making
Artificial Intelligence + Machine Learning:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=artificial-intelligence-machine-learning
AI for Healthcare and Life Sciences:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=ai-for-healthcare-and-life-sciences
Biological and Medical Devices and Systems:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=biological-and-medical-devices-and-systems
Graphics and Vision:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=graphics-and-vision
Human-Computer Interaction:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=human-computer-interaction
Multicore Processors & Cloud Computing:	 https://www.eecs.mit.edu/role/faculty-aid/?fwp_research=multicore-processors-cloud-computing
Security and

In [13]:
df_research = pd.DataFrame(data, columns=cols)

In [14]:
df_research

,research_group,url
0,AI and Society,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
1,Artificial Intelligence + Decision making,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
2,Artificial Intelligence + Machine Learning,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
3,AI for Healthcare and Life Sciences,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
4,Biological and Medical Devices and Systems,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
5,Graphics and Vision,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
6,Human-Computer Interaction,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
7,Multicore Processors & Cloud Computing,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
8,Security and Cryptography,https://www.eecs.mit.edu/role/faculty-aid/?fwp...
9,Systems and Networking,https://www.eecs.mit.edu/role/faculty-aid/?fwp...


In [15]:
df.to_csv(f"faculty-{SCHOOL}.csv", index=False)

In [16]:
# import xlsxwriter
file_xlsx = f"faculty-{SCHOOL}.xlsx"
writer = pd.ExcelWriter(file_xlsx, engine='xlsxwriter')
df.to_excel(writer, sheet_name = "CS Faculty-MIT", index=False)
df_research.to_excel(writer, sheet_name = "Research Groups", index=False)
writer.save()